## IgA-SeqPAT 16S OTU
Author: Victoria Ruiz & Thomas Battaglia

### Introduction
This Notebook is meant to contain the 16S_OTU data found within the respective manuscript. It contains only the code used to generate the figures found within the main text. This entire dataset is publically available in QIITA under the ID [10527](https://qiita.ucsd.edu/study/description/10527). More details about the procedures used to generate the data can be found within the **Methods** section of the manuscript. The table found within the data folder has been processed to remove any OTU less than 0.01% relative abudance.



### Generate Beta diversity PCoA's (Figure S3-a)
This command generates an HTML file that uses the program Emperor to view 3D PCoA data. The file `analysis/betadiv/unweighted_unifrac_emperor_pcoa_plot/index.html` is the output used to generate the figure within the manuscript.

In [13]:
# Make a folder to store the analysis
mkdir -p analysis/bdiv

# Run bdiv through plots
beta_diversity.py \
-i data/igaseq_otutable_n0001.biom \
-o analysis/bdiv \
-m unweighted_unifrac,weighted_unifrac \
-t data/rep_set.tre

In [14]:
# Generate PCoA from distance matrix
principal_coordinates.py \
-i analysis/bdiv/unweighted_unifrac_igaseq_otutable_n0001.txt \
-o analysis/bdiv/uw_bdiv_pcoa.txt

In [23]:
# Make an emperor 3-D file from PCoA
## Plot with custom axis
make_emperor.py \
-i analysis/bdiv/uw_bdiv_pcoa.txt \
-o analysis/bdiv/emperor_plot \
-m data/igaseq_mapping.txt \
--custom_axes Day_after_transfer \
--number_of_segments 14

### Generate Beta diversity PCoA's (Figure S3-b)
These commands will generate the graphs found within the multi-panel figure 3e. The command must be run on each of the 6 permutations. The final PDF is composed of multiple plots across the different Treatment groups and IgA fractions.

In [16]:
# Split otu-table by treatment and fractions
split_otu_table.py \
-i data/igaseq_otutable_n0001.biom \
-o data/per_study_otu_tables \
-m data/igaseq_mapping.txt \
-f Treatment,Fraction

In [17]:
# Make a folder to store the results
mkdir -p analysis/taxa_plots/

In [18]:
# Plot barplot with only Control-IgA-Negative
summarize_taxa_through_plots.py \
-i data/per_study_otu_tables/igaseq_otutable_n0001__Treatment_Control_Fraction_IgA-__.biom \
-o analysis/taxa_plots/control_negative \
-m data/per_study_otu_tables/igaseq_mapping__Treatment_Control_Fraction_IgA-__.txt \
-c Day_after_transfer_tissue \
-p scripts/parameters.txt \
--sort 

# Plot barplot with only Control-IgA-Positive
summarize_taxa_through_plots.py \
-i data/per_study_otu_tables/igaseq_otutable_n0001__Treatment_Control_Fraction_IgA+__.biom \
-o analysis/taxa_plots/control_positive \
-m data/per_study_otu_tables/igaseq_mapping__Treatment_Control_Fraction_IgA+__.txt \
-c Day_after_transfer_tissue \
-p scripts/parameters.txt \
--sort 

# Plot barplot with only Control-IgA-Total
summarize_taxa_through_plots.py \
-i data/per_study_otu_tables/igaseq_otutable_n0001__Treatment_Control_Fraction_Total__.biom \
-o analysis/taxa_plots/control_total \
-m data/per_study_otu_tables/igaseq_mapping__Treatment_Control_Fraction_Total__.txt \
-c Day_after_transfer_tissue \
-p scripts/parameters.txt \
--sort 

In [ ]:
# Plot barplot with only PAT-IgA-Negative
summarize_taxa_through_plots.py \
-i data/per_study_otu_tables/igaseq_otutable_n0001__Treatment_PAT_Fraction_IgA-__.biom \
-o analysis/taxa_plots/pat_negative \
-m data/per_study_otu_tables/igaseq_mapping__Treatment_PAT_Fraction_IgA-__.txt \
-c Day_after_transfer_tissue \
-p scripts/parameters.txt \
--sort 

# Plot barplot with only PAT-IgA-Positive
summarize_taxa_through_plots.py \
-i data/per_study_otu_tables/igaseq_otutable_n0001__Treatment_PAT_Fraction_IgA+__.biom \
-o analysis/taxa_plots/pat_positive \
-m data/per_study_otu_tables/igaseq_mapping__Treatment_PAT_Fraction_IgA+__.txt \
-c Day_after_transfer_tissue \
-p scripts/parameters.txt \
--sort 

# Plot barplot with only PAT-IgA-Total
summarize_taxa_through_plots.py \
-i data/per_study_otu_tables/igaseq_otutable_n0001__Treatment_PAT_Fraction_Total__.biom \
-o analysis/taxa_plots/pat_total \
-m data/per_study_otu_tables/igaseq_mapping__Treatment_PAT_Fraction_Total__.txt \
-c Day_after_transfer_tissue \
-p scripts/parameters.txt \
--sort 

### Differentially abundant taxa with LEfSe (Figure S3-d)
These commands will generate the raw data to be used to make the heatmap-style taxa list in the supplemental **Figure S1-d**. A custom tool called **Koeken** was developed to more easily run the **LEfSe** tool on the commands line, over multiple timepoints. Installation can be found [Koeken (Github)](https://github.com/twbattaglia/koeken). The resulting folder contains the intermediate file from running **LEfSe**. THe final table was imported into R and a heatmap was created using the **aheatmap** R library.

In [21]:
# Make directory to store results
mkdir -p analysis/lefse/

# Run Koeken between Control vs PAT1 pups
koeken.py \
-i data/igaseq_otutable_n0001.biom \
-o analysis/lefse/control_pat_igapos_overtime \
-m data/igaseq_mapping.txt \
--class Treat_Frac \
--split Day_after_transfer_tissue \
--compare Control_IgA+ PAT_IgA+ \
--level 7

In [22]:
# Beautify by collating to a heatmap-ish
pretty_lefse.py \
-i analysis/lefse/control_pat_igapos_overtime/lefse_output/run_lefse \
-o analysis/lefse/control_pat_igapos_overtime/heatmap/ \
-c Control_IgA+

### Average ICI score between IgA-postive and IgA-negative taxa (Figure S3-c)
